# Experimento com BERT

In [39]:
import pandas as pd

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [45]:
df = pd.read_csv("Dados/total_pares_smartphones_2.csv", dtype = {'ean_1': str, 'ean_2': str})
df.head(3)

,descricao_1,ean_1,titulo_1,url_1,titulo_cb_1,loja_1,descricao_2,ean_2,titulo_2,url_2,titulo_cb_2,loja_2,match
0,Sistema operacional Nome do sistema operaciona...,6934177738180,Pocophone Poco X3 Pro Dual sim 256GB Preto 8GB...,https://www.americanas.com.br/produto/4511090415,pocophone poco x3 pro dual sim 256gb preto 8gb...,americanas,NaN,7892509121040,"Smartphone Samsung Galaxy S21 FE 5G, Preto, 12...",https://www.amazon.com.br/Smartphone-Samsung-G...,"smartphone samsung galaxy s21 fe 5g, preto, 12...",amazon,0
1,Sobre este item Câmera dupla com sensor de ...,7892597350315,Smartphone Motorola Moto E7 32GB 2GB RAM Cinza...,https://www.amazon.com.br/Celular-Motorola-Mot...,smartphone motorola moto e7 32gb 2gb ram cinza...,amazon,Você merece ter um smartphone tecnológico e id...,7892597350315,Smartphone Motorola Moto E7 32GB 4G Wi-Fi Tela...,https://www.americanas.com.br/produto/2626417844,smartphone motorola moto e7 32gb 4g wi-fi tela...,americanas,1
2,O Smartphone Samsung Galaxy A12 conta com desi...,7892509116886,Smartphone Samsung Galaxy A12 64GB 4G Wi-Fi Te...,https://www.americanas.com.br/produto/2655733635,smartphone samsung galaxy a12 64gb 4g wi-fi te...,americanas,Sobre este item \r\n Isso se adeque às suas ...,5012786044210,"Motorola Baba Eletronica Ease34, Motorola, Branco",https://www.amazon.com.br/Motorola-Baba-Eletro...,"motorola baba eletronica ease34, motorola, branco",amazon,0


In [46]:
df.dtypes

descricao_1    object
ean_1          object
titulo_1       object
url_1          object
titulo_cb_1    object
loja_1         object
descricao_2    object
ean_2          object
titulo_2       object
url_2          object
titulo_cb_2    object
loja_2         object
match           int64
dtype: object

In [47]:
df.shape

(160, 13)

In [48]:
df[df["titulo_1"]==df["titulo_2"]]

,descricao_1,ean_1,titulo_1,url_1,titulo_cb_1,loja_1,descricao_2,ean_2,titulo_2,url_2,titulo_cb_2,loja_2,match
155,NaN,7892509120623,Smartphone Samsung Galaxy A52s 5G 128GB 6.5 6G...,https://www.amazon.com.br/Smartphone-Galaxy-A5...,smartphone samsung galaxy a52s 5g 128gb 6.5 6g...,amazon,Características: - Marca: Samsung - Modelo: A5...,7892509120623,Smartphone Samsung Galaxy A52s 5G 128GB 6.5 6G...,https://www.americanas.com.br/produto/4472100131,smartphone samsung galaxy a52s 5g 128gb 6.5 6g...,americanas,1


## Otimizador e Inicialização do Modelo

In [49]:
test = None

# can be up to 512 for BERT
MAX_LENGTH = 256
BATCH_SIZE = 32
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case = True)

#######################INICIOS FUNÇÕES DE APOIO#######################

def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label


def convert_example_to_feature(titulo_1, titulo_2):
    return tokenizer.encode_plus(titulo_1, titulo_2,
                                 add_special_tokens = True, # adiciona [CLS], [SEP]
                                 max_length = MAX_LENGTH, # comprimento máximo do texto de entrada
                                 padding = 'max_length', # adiciona [PAD] até o tam_max (MAX_LENGTH)
                                 truncation = True, # padrão = 'longest_first'
                                 return_attention_mask = True, # adiciona máscara de atenção para não focar nos tokens do pad
                                )

def encode_examples(df_titulos, labels, limit = -1):
    
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    
    if (limit > 0):
        ds = ds.take(limit)
    
    # for review, label in tfds.as_numpy(ds):
    for titulo_1, titulo_2, label in zip(df_titulos["titulo_1"], df_titulos["titulo_2"], labels):
        
        bert_input = convert_example_to_feature(titulo_1, titulo_2)
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
        
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

def get_bert_data(X_train, y_train, X_valid, y_valid, X_test, y_test):
    
    # train dataset
    ds_train = encode_examples(X_train, y_train).batch(BATCH_SIZE)

    # test dataset
    ds_test = encode_examples(X_test, y_test).batch(BATCH_SIZE)

    #validation dataset
    ds_valid = encode_examples(X_valid, y_valid).batch(BATCH_SIZE)

    return ds_train, ds_valid, ds_test

#######################FIM FUNÇOES DE APOIO#######################

def get_test_metrics(model, ds_test, y_test):

    #Predictin test dataset
    tf_output = model.predict(ds_test)[0]
    tf_prediction = tf.nn.softmax(tf_output, axis=1)
    # labels = ['Negative','Positive'] #(0:negative, 1:positive)
    label = tf.argmax(tf_prediction, axis=1)
    label_pred = label.numpy()
    # print(label_pred)

    print(classification_report(y_test, label_pred))

    print(confusion_matrix(y_test, label_pred))

    return label_pred


def pipeline_bert(name, X_train, y_train, X_valid, y_valid, X_test, y_test): #X_train = [titulos1, titulos2]

    learning_rate = 2e-5
    number_of_epochs = 3
    ds_train, ds_valid, ds_test = get_bert_data(X_train, y_train, X_valid, y_valid, X_test, y_test)
    
    # model initialization
    model = TFBertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', from_pt = True)

    # choosing Adam optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric_acc = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric_acc])

    #Training model
    bert_history = model.fit(ds_train, epochs=number_of_epochs, validation_data=ds_valid)
    
    #Predict test data
    result = get_test_metrics(model, ds_test, y_test)
    return result

    # metrics = calc_metrics(y_test, result, name)
    # return metrics
    

In [50]:
X = df[["titulo_1", "titulo_2"]]#[df["titulo_1"].to_list(), df["titulo_2"].to_list()]
y = df["match"].to_list()

X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.2, random_state = 42, stratify=y_train_valid)


In [52]:
pipeline_bert("name", X_train, y_train, X_valid, y_valid, X_test, y_test)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
3/3 [==============================] - 111s 32s/step - loss: 0.6765 - accuracy: 0.5393 - val_loss: 0.5847 - val_accuracy: 0.7826
Epoch 2/3
3/3 [==============================] - 92s 31s/step - loss: 0.5604 - accuracy: 0.8989 - val_loss: 0.4277 - val_accuracy: 1.0000
Epoch 3/3
3/3 [==============================] - 103s 36s/step - loss: 0.4479 - accuracy: 0.9438 - val_loss: 0.3261 - val_accuracy: 1.0000
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        24
           1       0.92      1.00      0.96        24

    accuracy                           0.96        48
   macro avg       0.96      0.96      0.96        48
weighted avg       0.96      0.96      0.96        48

[[22  2]
 [ 0 24]]


array([1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0], dtype=int64)